<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Similarity-interface" data-toc-modified-id="Similarity-interface-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Similarity interface</a></span><ul class="toc-item"><li><span><a href="#LSI-生成向量" data-toc-modified-id="LSI-生成向量-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>LSI 生成向量</a></span></li><li><span><a href="#余弦向量比较相似性" data-toc-modified-id="余弦向量比较相似性-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>余弦向量比较相似性</a></span></li><li><span><a href="#初始化query-结构模型" data-toc-modified-id="初始化query-结构模型-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>初始化query 结构模型</a></span><ul class="toc-item"><li><span><a href="#Warning" data-toc-modified-id="Warning-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Warning</a></span></li><li><span><a href="#导入和导出-index" data-toc-modified-id="导入和导出-index-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>导入和导出 index</a></span></li></ul></li><li><span><a href="#查找相似性--利用index" data-toc-modified-id="查找相似性--利用index-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>查找相似性--利用index</a></span></li><li><span><a href="#进行降序排列" data-toc-modified-id="进行降序排列-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>进行降序排列</a></span></li></ul></li></ul></div>

In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


# Similarity interface
* similarity between pairs of document

In [2]:
from gensim import corpora, models, similarities

dictionary = corpora.Dictionary.load('deerwester.dict')
corpus = corpora.MmCorpus('corpus.mm')
print(corpus)

2018-12-25 00:20:38,690 : INFO : 'pattern' package not found; tag filters are not available for English
2018-12-25 00:20:38,698 : INFO : loading Dictionary object from deerwester.dict
2018-12-25 00:20:38,699 : INFO : loaded deerwester.dict
2018-12-25 00:20:38,701 : INFO : loaded corpus index from corpus.mm.index
2018-12-25 00:20:38,702 : INFO : initializing cython corpus reader from corpus.mm
2018-12-25 00:20:38,704 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


MmCorpus(9 documents, 12 features, 28 non-zero entries)


In [3]:
list(corpus)

[[(0, 1.0), (1, 1.0), (2, 1.0)],
 [(0, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0)],
 [(2, 1.0), (5, 1.0), (7, 1.0), (8, 1.0)],
 [(1, 1.0), (5, 2.0), (8, 1.0)],
 [(3, 1.0), (6, 1.0), (7, 1.0)],
 [(9, 1.0)],
 [(9, 1.0), (10, 1.0)],
 [(9, 1.0), (10, 1.0), (11, 1.0)],
 [(4, 1.0), (10, 1.0), (11, 1.0)]]

## LSI 生成向量

In [4]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

2018-12-25 00:21:49,386 : INFO : using serial LSI version on this node
2018-12-25 00:21:49,387 : INFO : updating model with new documents
2018-12-25 00:21:49,389 : INFO : preparing a new chunk of documents
2018-12-25 00:21:49,390 : INFO : using 100 extra samples and 2 power iterations
2018-12-25 00:21:49,391 : INFO : 1st phase: constructing (12, 102) action matrix
2018-12-25 00:21:49,392 : INFO : orthonormalizing (12, 102) action matrix
2018-12-25 00:21:49,403 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2018-12-25 00:21:49,406 : INFO : computing the final decomposition
2018-12-25 00:21:49,407 : INFO : keeping 2 factors (discarding 43.156% of energy spectrum)
2018-12-25 00:21:49,409 : INFO : processed documents up to #9
2018-12-25 00:21:49,411 : INFO : topic #0(3.341): 0.644*"system" + 0.404*"user" + 0.301*"eps" + 0.265*"response" + 0.265*"time" + 0.240*"computer" + 0.221*"human" + 0.206*"survey" + 0.198*"interface" + 0.036*"graph"
2018-12-25 00:21:49,412 : INFO : topic #1(2

In [5]:
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())

In [6]:
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[(0, 0.4618210045327161), (1, 0.07002766527900028)]


## 余弦向量比较相似性

## 初始化query 结构模型

In [7]:
index = similarities.MatrixSimilarity(lsi[corpus]) # transform corpus to LSI space and index it

2018-12-25 00:29:36,282 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2018-12-25 00:29:36,284 : INFO : creating matrix with 9 documents and 2 features
/Users/liuchuang/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


### Warning

* similarities.MatrixSimilarity : 只适合所有的向量都放进内存里面
    
* 如果内存不足，similarities.Similarity class： 在固定的内存上操作，分割成多个文件

### 导入和导出 index

In [8]:
index.save('deerwester.index')
index = similarities.MatrixSimilarity.load('deerwester.index')

2018-12-25 00:35:07,183 : INFO : saving MatrixSimilarity object under deerwester.index, separately None
2018-12-25 00:35:07,185 : INFO : saved deerwester.index
2018-12-25 00:35:07,185 : INFO : loading MatrixSimilarity object from deerwester.index
2018-12-25 00:35:07,186 : INFO : loaded deerwester.index


## 查找相似性--利用index

In [9]:
sims = index[vec_lsi] # perform a similarity query against the corpus
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

[(0, 0.998093), (1, 0.93748635), (2, 0.9984453), (3, 0.9865886), (4, 0.90755945), (5, -0.12416792), (6, -0.10639259), (7, -0.09879464), (8, 0.050041765)]


## 进行降序排列

In [10]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims) # print sorted (document number, similarity score) 2-tuples

[(2, 0.9984453), (0, 0.998093), (3, 0.9865886), (1, 0.93748635), (4, 0.90755945), (8, 0.050041765), (7, -0.09879464), (6, -0.10639259), (5, -0.12416792)]


* 可以看出 分值越大 主题越符合